# PH Lesson

- What to expect
- What is in the corpus?
  - Vasif
- Why

## Text Extraction

- ABBYY FineReader -> HTML
- Alternative: just getting the text file: hard to split!
- Alternative: Gemini 2.5: technology not there yet

Here is what a PDF looks like

Here is the HTML
    some code about how I worked with the html to extract text
    Splitting text into years and the years into subheadings

Is there a way to do this with beautiful soup?

In [ ]:
import os
import re
import pandas as pd
import csv
import numpy as np

In [ ]:
def configure_for_text_file(text_file_name):
    config = {}
    if text_file_name == 'ahmedcevdet_17741783_ttk_ft.txt':
        config['year_regex'] = '<p><span class=\"font4\" style=\"font-weight:bold;\">.*?\d{4}</span></p>'
        config['subheading_regex'] = '<p><span class=\"font4\" style=\"font-weight:bold;\">.*?</span></p>'

    elif text_file_name == 'ahmedcevdet_17831786_ttk_ft.txt':
        config['year_regex'] = '<h1>.*?\d{4}.*?</h1>'
        config['subheading_regex'] = '<h2><a name=".*"></a><span class="font6" style="font-weight:bold;">.*</span></h2>'

    elif text_file_name == 'enveri_17741783_ft.txt':
        config['year_regex'] = '\(\d{4}\/1'
        config['subheading_regex'] = '<h5><a name="bookmark\\d+"></a><span class="font13" style="font-weight:bold;">(?:<a name="bookmark\\d+"></a>)*.*?</span></h5>'

    elif text_file_name == 'enveri_17681774_ft.txt':
        config['year_regex'] = '(?:(?:<p><span class=\"font21\" style=\"font-weight:bold;\">)VEK.+(?:</span></p>))'
        config['subheading_regex'] = '(?:(?:<p><span class=\"font21\" style=\"font-weight:bold;\">).+(?:</span></p>))'

    elif text_file_name == 'vasif_17521774_ft.txt':
        config['year_regex'] = '<(?:p|h2)>.*<span class="font15" style="font-weight:bold;">(?:<a name="bookmark\d+"></a>)*VEK.*?</(?:p|h2)>'
        config['subheading_regex'] = '<h2><a name="bookmark\d+"></a><span class="[^"]+" style="[^"]+"><a name="bookmark\d+"></a>(.*?)</span></h2>'

    elif text_file_name == 'vasif_17741779_ft.txt':
        config['year_regex'] = '<p><span class="font15" style="font-weight:bold;">Vek.*</span></p>'
        config['subheading_regex'] = '(?:<p><span class="font15" style="font-weight:bold;">.*</span></p>|<h3><a name="bookmark\d+"></a><span class="font15" style="font-weight:bold;"><a name="bookmark\d+"></a>.*</span></h3>)'

    elif text_file_name == 'vasif_17821787_ft.txt':
        config['year_regex'] = '<p><span class="font21" style="font-style:italic;">Vek.*</span></p>'
        config['subheading_regex'] = '<h6><a name="bookmark\d+"></a><span class="font21" style="font-weight:bold;"><a name="bookmark\d+"></a>.*</span></h6>'

    elif text_file_name == 'vasif_17941805_ft.txt':
        config['year_regex'] = '<p><span class="font11" style="font-weight:bold;">VEK.*</span></p>'
        config['subheading_regex'] = '<h2><a name="bookmark\d+"></a><span class="font11" style="font-weight:bold;"><a name="bookmark\d+"></a>.*</span></h2>'

    elif text_file_name == 'nuri_17941799_ft.txt':
        config['year_regex'] = '<(?:p|h2)>.*\[1\d{3}\].*</(?:p|h2)>'
        config['subheading_regex'] = '<(?:p|h2)>.*<span class="font11" style="font-weight:bold;">.*</(?:p|h2)>'

    return config

In [ ]:
def divide_file_by_year(text, year_regex, output_directory):
    year_matches = re.finditer(year_regex, text)

    year_breaks = [match.span() for match in year_matches]

    if len(year_breaks) == 0:
        print(f"Error with year_regex {output_directory}")
        return

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Handle introduction section
    intro_end = year_breaks[0][0] if year_breaks else len(text)
    intro_content = text[:intro_end]

    intro_path = os.path.join(output_directory, "introduction")
    os.makedirs(intro_path, exist_ok=True)
    with open(os.path.join(intro_path, "introduction.txt"), 'w') as intro_file:
        intro_file.write(intro_content)

    year_counter = 1
    for i in range(len(year_breaks)):
        start_index, _ = year_breaks[i]
        end_index = year_breaks[i + 1][0] if i + 1 < len(year_breaks) else len(text)

        year_content = text[start_index:end_index]

        # Extract year for folder naming
        year_str = text[year_breaks[i][0]:year_breaks[i][1]]
        if 'VEK' in year_str or 'Vek' in year_str:
            # Enumerate year if VEK or Vek is found
            year_folder_name = f"Year_{year_counter}"
            year_counter += 1
        else:
            year_search = re.search(r'\d{4}', year_str)
            if year_search:
                year_folder_name = year_search.group()
            else:
                print(f"Year not found in text: {year_str}")

        year_path = os.path.join(output_directory, year_folder_name)
        os.makedirs(year_path, exist_ok=True)
        with open(os.path.join(year_path, year_folder_name + ".txt"), 'w') as year_file:
            year_file.write(year_content)

In [ ]:
def divide_file_by_subheadings(file_path, subheading_regex, output_directory, year):
    with open(file_path, 'r') as f:
        text = f.read()

        subheading_matches = re.finditer(subheading_regex, text)
        subheading_breaks = [match.span() for match in subheading_matches]

        if not subheading_breaks:
            print(f"No subheadings found in {file_path}")
            return

        # Handle initial segment before the first subheading
        start_index = 0
        end_index = subheading_breaks[0][0]
        initial_segment = text[start_index:end_index]
        with open(os.path.join(output_directory, "{}_1.txt".format(year)), 'w') as segment_file:
            segment_file.write(initial_segment)

        # Start from segment number 2 for subsequent subheadings
        segment_number = 2

        for start, end in subheading_breaks:
            start_index = start
            end_index = subheading_breaks[segment_number-1][0] if segment_number-1 < len(subheading_breaks) else len(text)

            subheading_content = text[start_index:end_index]

            # Format file name: year + segment number
            segment_filename = "{}_{}.txt".format(year, segment_number)
            segment_filename = re.sub('[^\w\-_\. ]', '_', segment_filename)  # Sanitize filename

            with open(os.path.join(output_directory, segment_filename), 'w') as segment_file:
                segment_file.write(subheading_content)

            segment_number += 1

In [ ]:
def clean_text(text):

    # Remove HTML tags
    text = re.sub(r'\<[^>]*\>', '', text)

    # These patterns and texts were determined with Umar for the initial fulltext
    # extraction and cleaning

    regex_patterns = [
        '\([0-9]{1,3}\s?[ab]\)', '(\(|\[)[0-9]{1,3}\-[ab](\)|\])', '\(V[arkARK]+.+\)',
        '\-[0-9]{1,3}[ab]\-', '\([Ss]\.\s?[0-9]{1,3}\)', '\[\s[0-9]{1,3}\s\]',
        '\[[0-9]{1,3}[abAB]\]', '[0-9]{1,3}[ab]', '\(\s*\)',
        '\(SAFYA\s([0-9]{1,3}|&amp;)\)', '\[M1\s[0-9]{1,3}\]',
        '(\[Ü[12]\s*\]|\[Ü[1]\s*\-\s*Ü[2]\s*\])', '\([0-9]{1,3}\)',
        '\<a\sname=\"bookmark[0-9]+\"\>', '\<sup\>[0-9]*'
    ]
    texts_to_remove = [
        '&nbsp;', '&quot;', "(silik)", "(?)", "■", "(İla)", "(la)", "(lb)", "#",
        "&amp;", "[lb]"
    ]

    # Apply regex replacements
    for pattern in regex_patterns:
        text = re.sub(pattern, "", text)

    # Apply text replacements
    for remove_text in texts_to_remove:
        text = text.replace(remove_text, '')

    return text

In [ ]:
def clean_files_in_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()

                cleaned_content = clean_text(content)

                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(cleaned_content)

In [ ]:
import shutil

def move_original_year_files_to_folder(year_folder_path, output_directory):
    # Determine the name of the year file
    year_file_name = os.path.basename(year_folder_path) + ".txt"
    year_file_path = os.path.join(year_folder_path, year_file_name)

    # Define the new directory to store the year files
    new_directory = output_directory + "yearfiles/"
    os.makedirs(new_directory, exist_ok=True)

    # Define the new file path in the year files directory
    new_file_path = os.path.join(new_directory, year_file_name)

    # Move the file if it exists
    if os.path.exists(year_file_path):
        try:
            shutil.move(year_file_path, new_file_path)
        except Exception as e:
            print(f"Error moving file {year_file_path} to {new_file_path}: {e}")


In [ ]:
def process_text_file(text_file_name, work_directory, output_directory):
    config = configure_for_text_file(text_file_name)

    with open(os.path.join(work_directory, text_file_name), 'r', encoding='utf-8') as f:
        text = f.read()

    divide_file_by_year(text, config['year_regex'], output_directory)

    year_folders = os.listdir(output_directory)
    for folder in year_folders:
        year_folder_path = os.path.join(output_directory, folder)
        year_file_path = os.path.join(year_folder_path, folder + ".txt")
        divide_file_by_subheadings(year_file_path, config['subheading_regex'], year_folder_path, folder)
        clean_files_in_directory(year_folder_path)
        move_original_year_files_to_folder(year_folder_path, output_directory)


In [ ]:
# vasif_17521774
text_file_name = 'vasif_17521774_ft.txt'
work_directory = "html_for_extraction/"
output_directory = 'vasif_17521774/'
process_text_file(text_file_name, work_directory, output_directory)

Reflect on Kocasekbanbasi and the Gemini 2.5 differences

## Text Normalization

## Topic Modeling